# Neural Networks with Sequential and Functional API

In [1]:
import tensorflow as tf
from tensorflow import keras # the official higher level API
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)

x_train = x_train.reshape(-1, 28*28).astype('float32') / 255.0 # Keep 60000, flatten, between 0 - 1
x_test = x_test.reshape(-1, 28*28).astype('float32') / 255.0
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(60000,)
(60000, 784)
(10000, 784)


In [13]:
# Method 1 - Sequential API (Very convenient, not very flexible)
model = keras.Sequential(
    [   
        keras.Input(shape=(28*28)),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(10),
    ]
)

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), # cause we never specify softmax
    optimizer = keras.optimizers.Adam(lr=0.001),
    metrics=['accuracy'],
)

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_10 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_11 (Dense)             (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


In [14]:
# 2nd Method
model = keras.Sequential()
model.add(keras.Input(shape=(784)))
model.add(layers.Dense(512, activation='relu'))
#print(model.summary()) # a form of debugging with this 2nd method
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10))

model = keras.Model(inputs = model.inputs,
                    outputs=[model.layers[-2].output])

feature = model.predict(x_train)
print(feature.shape)

(60000, 256)


In [11]:
# Functional API (A bit more flexible)
inputs = keras.Input(shape=(784))
x = layers.Dense(512, activation='relu', name='first layer')(inputs)
x = layers.Dense(256, activation='relu', name='second layer')(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False), # we specified softmax
    optimizer = keras.optimizers.Adam(lr=0.001),
    metrics = ['accuracy']
)

In [12]:
model.fit(x_train, y_train, batch_size=32, epochs=5)
model.evaluate(x_test, y_test, batch_size=32)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1863 - accuracy: 0.9426
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0785 - accuracy: 0.9758
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0539 - accuracy: 0.9833
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0421 - accuracy: 0.9865
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.1101 - accuracy: 0.9722


[0.11010511219501495, 0.9721999764442444]